Import necessary libraries:

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!wget https://raw.githubusercontent.com/towardsai/tutorials/master/decision_tree_learning/Iris.csv

--2022-11-27 16:15:09--  https://raw.githubusercontent.com/towardsai/tutorials/master/decision_tree_learning/Iris.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5103 (5.0K) [text/plain]
Saving to: ‘Iris.csv.1’

Iris.csv.1          100%[===================>]   4.98K  --.-KB/s    in 0s      

2022-11-27 16:15:09 (50.0 MB/s) - ‘Iris.csv.1’ saved [5103/5103]



Column names and dataset shape


In [ ]:
col_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'specie']
data = pd.read_csv('Iris.csv' ,skiprows=1, header=None, names=col_names)
data

,sepal_length,sepal_width,petal_length,petal_width,specie
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
146,6.7,3.0,5.2,2.3,Iris-virginica
147,6.3,2.5,5.0,1.9,Iris-virginica
148,6.5,3.0,5.2,2.0,Iris-virginica
149,6.2,3.4,5.4,2.3,Iris-virginica


Change specie column into values

In [ ]:
from sklearn.preprocessing import LabelEncoder
Le = LabelEncoder()

data['specie'] = Le.fit_transform(data['specie'])
data


,sepal_length,sepal_width,petal_length,petal_width,specie
1,5.1,3.5,1.4,0.2,0
2,4.9,3.0,1.4,0.2,0
3,4.7,3.2,1.3,0.2,0
4,4.6,3.1,1.5,0.2,0
5,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
146,6.7,3.0,5.2,2.3,2
147,6.3,2.5,5.0,1.9,2
148,6.5,3.0,5.2,2.0,2
149,6.2,3.4,5.4,2.3,2


Decision Tree Algorithm

In [ ]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        '''Constructor''' 
        
        # Decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        # leaf node
        self.value = value

In [ ]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        
        #initializing the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        '''Recursive function for bulding the tree''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split the tree until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split["info_gain"]>0:
                left_out_tree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_out_tree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_out_tree, right_out_tree, best_split["info_gain"])
        
        # computing the leaf node
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' A function to find the best split possible '''
        
        best_split = {}
        max_info_gain = -float("inf")
        
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        '''function to split the data'''
        
        dataset_l = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_r = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_l, dataset_r
    
    def information_gain(self, parent, left_child, right_child, mode="entropy"):
        ''' function for information gain '''
        
        weight_left = len(left_child) / len(parent)
        weight_right = len(right_child) / len(parent)
        if mode=="gini":
            info_gain = self.gini_index(parent) - (weight_left*self.gini_index(left_child) + weight_right*self.gini_index(right_child))
        else:
            info_gain = self.entropy(parent) - (weight_left*self.entropy(left_child) + weight_right*self.entropy(right_child))
        return info_gain
    
    def entropy(self, y):
        ''' function for entropy'''
        
        class_labels = np.unique(y)
        entropy = 0
        for cis in class_labels:
            p_cis = len(y[y == cis]) / len(y)
            entropy += -p_cis * np.log2(p_cis)
        return entropy
    
    def gini_index(self, y):
        ''' function for Gini Index '''
        
        class_labels = np.unique(y)
        gini_value = 0
        for cls in class_labels:
            p_cis = len(y[y == cls]) / len(y)
            gini_value += p_cis**2
        return 1 - gini_value
        
    def calculate_leaf_value(self, Y):
        ''' function for calculating leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree'''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft side:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright side:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [ ]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [ ]:

classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train,Y_train)
classifier.print_tree()


X_2 <= 1.9 ? 0.33741385372714494
 left side:0.0
 right side:X_3 <= 1.5 ? 0.427106638180289
  left side:X_2 <= 4.9 ? 0.05124653739612173
    left side:1.0
    right side:2.0
  right side:X_2 <= 5.0 ? 0.019631171921475288
    left side:X_1 <= 2.8 ? 0.20833333333333334
        left side:2.0
        right side:1.0
    right side:2.0


In [ ]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.9333333333333333